In [ ]:
!pip install llama-index
!pip install openai
!pip install sentence-transformers
!pip install pdf2image
!pip install Flask

In [ ]:
from flask import Flask

app = Flask(__name__)


@app.route("/")
def home():
    return "Hello World!"


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5601)

In [ ]:
from llama_index.indices.loading import load_index_from_storage
import os
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, StorageContext

os.environ["OPENAI_API_KEY"] = "xxx"

index = None
index_dir = "./storage"


def initialize_index():
    global index
    if os.path.exists(index_dir):
        storage_context = StorageContext.from_defaults(persist_dir=index_dir)
        index = load_index_from_storage(storage_context=storage_context)
    else:
        storage_context = StorageContext.from_defaults()
        documents = SimpleDirectoryReader(
            "/content/drive/MyDrive/Colab Notebooks/data/documents"
        ).load_data()
        index = GPTVectorStoreIndex.from_documents(
            documents=documents, storage_context=storage_context
        )
        storage_context.persist(index_dir)

In [ ]:
from werkzeug.utils import secure_filename
from flask import request
from multiprocessing.managers import BaseManager
import os

manager = BaseManager(("", 5602), b"password")
manager.register("query_index")
manager.connect()


@app.route("/query", methods=["GET"])
def query_index():
    query_text = request.args.get("text", None)
    if not query_text:
        return "No text found, please include a ?text=blah parameter in the URL", 400
    response = manager.query_index(query_text)._getvalue()
    return response, 200


manager.register("insert_init_index")


@app.route("/uploadFile", method=["POST"])
def upload_file():
    global manager
    if "file" not in request.files:
        return "Please send a POST request with a file", 400
    filepath = None
    try:
        uploaded_file = request.files["file"]
        filename = secure_filename(uploaded_file)
        filepath = os.path.join("documents", os.path.basename(filename))
        uploaded_file.save(filepath)

        if request.form.get("filename_as_doc_id", None) is not None:
            manager.insert_into_index(filepath, doc_id=filename)
        else:
            manager.insert_into_index(filepath)
    except Exception as e:
        if filepath is not None and os.path.exists(filepath):
            os.remove(filepath)
        return "Error: {}".format(str(e)), 500

    if filepath is not None and os.path.exists(filepath):
        os.remove(filepath)

    return "File inserted!", 200

In [ ]:
import os
from multiprocessing import Lock
from multiprocessing.managers import BaseManager
from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex, Document

index = None
lock = Lock()


def query_index(query_text):
    global index
    query_engine = index.as_query_engine()
    response = query_engine.query(query_text)
    return str(response)


def insert_into_index(doc_text, doc_id=None):
    global index
    document = SimpleDirectoryReader(input_files=[doc_text]).load_data()[0]
    if doc_id is not None:
        document.doc_id = doc_id

    with lock:
        index.insert(document)
        index.storage_context.persist()


def initialize_index():
    global index
    with lock:
        if os.path.exists(index_dir):
            storage_context = StorageContext.from_defaults(persist_dir=index_dir)
            index = load_index_from_storage(storage_context=storage_context)
        else:
            storage_context = StorageContext.from_defaults()
            documents = SimpleDirectoryReader(
                "/content/drive/MyDrive/Colab Notebooks/data/documents"
            ).load_data()
            index = GPTVectorStoreIndex.from_documents(
                documents=documents, storage_context=storage_context
            )
            storage_context.persist(index_dir)


if __name__ == "__main__":
    initialize_index()

    manager = BaseManager(("", 5602), b"password")
    manager.register("query_index", query_index)
    manager.register("insert_into_index", insert_into_index)
    server = manager.get_server()
    server.serve_forever()